# Workflow & Hyperparameter Optimization

In [56]:
import pandas as pd
import seaborn as sns
import numpy as np

👇 Import the house price data set. We will only keep numerical feature for sake of simplicity

Your goal will be to fit the best KNN Regressor. And in particular, how many "neighbors" (K in KNN) should you consider to best predict your house-price?

In [57]:
# Load raw data
data = pd.read_csv('https://wagon-public-datasets.s3.amazonaws.com/houses_train_raw.csv')

# Only keep numerical columns and raws without NaN
data = data.select_dtypes(include=np.number).dropna()

data

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,61,0,0,0,0,0,2,2008,208500
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,298,0,0,0,0,0,0,5,2007,181500
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0,42,0,0,0,0,0,9,2008,223500
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0,35,272,0,0,0,0,2,2006,140000
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,192,84,0,0,0,0,0,12,2008,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,62.0,7917,6,5,1999,2000,0.0,0,...,0,40,0,0,0,0,0,8,2007,175000
1456,1457,20,85.0,13175,6,6,1978,1988,119.0,790,...,349,0,0,0,0,0,0,2,2010,210000
1457,1458,70,66.0,9042,7,9,1941,2006,0.0,275,...,0,60,0,0,0,0,2500,5,2010,266500
1458,1459,20,68.0,9717,5,6,1950,1996,0.0,49,...,366,0,112,0,0,0,0,4,2010,142125


In [58]:
X = data.drop(columns=['SalePrice'])
y = data['SalePrice']

## 1. Train/Test split

👇 Split the data to create your `X_train` `X_test` and `y_train` `y_test`. Use:
- `test_size=0.3`
- `random_state=0` to compare with your buddy

In [60]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.3, random_state=0) 

## 2. Scaling

Scaling is always very important for KNN.

❓ _Standard-Scale_ your training set

In [61]:
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler()
X_scaled = X_scaler.fit_transform(X_train)
X_scaled
#data leakage 
#test juste un transform 

array([[-1.37511158, -0.87871936,  0.18688486, ..., -0.14608468,
        -0.11071095,  0.16828509],
       [-1.30912075, -0.87871936,  2.10800891, ..., -0.14608468,
        -1.96535592, -0.57711328],
       [ 0.57633154, -0.87871936, -0.45348983, ..., -0.14608468,
         0.26021804, -0.57711328],
       ...,
       [ 0.88978798, -0.87871936,  0.40034309, ...,  3.97498424,
        -1.22349793, -1.32251165],
       [ 0.04133445, -0.14400627, -0.45348983, ..., -0.14608468,
        -0.48163994,  0.16828509],
       [ 0.41135447, -0.87871936, -0.45348983, ..., -0.14608468,
        -0.11071095, -0.57711328]])

## 3. Baseline KNN model

❓ 5-fold cross validate a simple KNN regressor taking into account only the closest neighbor, and compute its mean cv-score

In [129]:
from sklearn.model_selection import cross_validate
from sklearn.neighbors import KNeighborsRegressor

model = KNeighborsRegressor(n_neighbors=1)
resc_score = cross_validate(model,X_scaled,y_train,cv=5, 
                            scoring=['r2'])

r2 = resc_score['test_r2'].mean()
r2

0.569025195507008

## 4. Grid search

Let's use sklearn `GridSearchCV` to find the best KNN hyperparameter `n_neighbors`.
- Start coarse-grain approach, with `n_neighbors` = [1,5,10,20,50]
- 5-fold cross validate each combination
- Be sure to maximize your performance time using `n_jobs`

In [68]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
# Train/Test split
# Instanciate model
model = KNeighborsRegressor()
# Hyperparameter Grid
grid = {'n_neighbors': [1,5,10,20,50]}

# Instanciate Grid Search
search = GridSearchCV(model, grid, 
                           scoring = 'r2',
                           cv = 5,
                           n_jobs=-1 # paralellize computation
                          ) 

# Fit data to Grid Search
search.fit(X_scaled,y_train);

#np.dot arrange 
#np linalg
#np log space 

❓ According to the grid search, what is the optimal K value?

In [69]:
search.best_params_ 

{'n_neighbors': 20}

❓ What is the best score the optimal K value produced?

In [70]:
search.best_score_

0.7594265833471792

We now have an idea about where the best k lies, but some of the values we did not try could be better!

Re-run a fine-grain grid search with k values around to your previous best value

❓ What is the `best_score` and `best_k` you find?

In [72]:
best_score = 0.77
best_k = 20

#### 🧪 Test your code

In [73]:
from nbresult import ChallengeResult
result = ChallengeResult('knn',
                         best_k=best_k,
                         best_score=best_score)
result.write()
print(result.check())

============================= test session starts ==============================
platform darwin -- Python 3.8.12, pytest-6.2.5, py-1.10.0, pluggy-1.0.0 -- /Users/selmalopez/.pyenv/versions/lewagon_current/bin/python3
cachedir: .pytest_cache
rootdir: /Users/selmalopez/code/selmalopez/data-challenges/05-ML/05-Model-Tuning/01-Workflow
plugins: dash-2.0.0, anyio-3.3.2
collecting ... collected 2 items

tests/test_knn.py::TestKnn::test_best_k PASSED                           [ 50%]
tests/test_knn.py::TestKnn::test_best_score PASSED                       [100%]

============================== 2 passed in 0.05s ===============================


💯 You can commit your code:

git add tests/knn.pickle

git commit -m 'Completed knn step'

git push origin master


### Visual check

☝️ This problem is actually simple enough to perform a grid search manually.
- Loop manually over all values of k from 1 to 50 and store the mean cv-scores of each model in a list.
- Plot the score as a function of k to visualy find the best k

In [76]:
list_i = []
list_nei = []
for i in range(1,51):
    model = KNeighborsRegressor(n_neighbors=i)
    cv_results = cross_validate(model, X_scaled, y_train, cv=5,
                                scoring=['r2'])
    closest_nei = cv_results['test_r2'].mean()
    list_nei.append(closest_nei)
list_nei

[0.569025195507008,
 0.7048465720721123,
 0.7148574162449645,
 0.7369589894410885,
 0.7398576398909773,
 0.7561726759876845,
 0.7570719839640191,
 0.7575426569399312,
 0.7560552108529445,
 0.7582358201800268,
 0.7563805458508698,
 0.7626395621015083,
 0.7646979094747561,
 0.7655062000041237,
 0.7636794502232177,
 0.7619082979607626,
 0.7617903771847266,
 0.7595089732516886,
 0.7592645274173245,
 0.7594265833471792,
 0.7605446827763104,
 0.7589734692315226,
 0.7608013468561843,
 0.7592411296816367,
 0.7592097815174718,
 0.7570561296556418,
 0.7548268303731842,
 0.7540504424580443,
 0.7506512644236315,
 0.7495364715241382,
 0.7466270036785387,
 0.7451801083541939,
 0.7437972769220396,
 0.743665108088938,
 0.7432249120642472,
 0.7406588707058075,
 0.7393818664820948,
 0.7381614076927718,
 0.7363705356192449,
 0.7356373254215228,
 0.7340147270090075,
 0.7340705722415277,
 0.7318262974576847,
 0.7301822632008869,
 0.7291261510658155,
 0.7282291357063732,
 0.7275856292661133,
 0.726872685715

❓Can you guess what makes GridSearchCV a better option than such manual loop ?
 
<details>
    <summary>Answer</summary>

- Sklearn's `n_jobs=-1` allows you to paralellize search of each CPU
- What if you had multiple hyper-parameters to co-optimize ?
</details>

## 5. Multiple params

KNNRegressor suppports various _distance metrics_ via the hyper-parameter `p` [see docs](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html)

❓Use GridSearchCV to search for best `k` and `p` at the same time: Try all combinations for `k` = [1, 5, 10, 20, 50] and `p` = [1, 2, 3]. 

In [113]:
# Hyperparameter Grid
grid = {'n_neighbors': [1, 5, 10, 20, 50] , 
        'p': [1, 2, 3]}

# Instanciate Grid Search
search1 = GridSearchCV(model, grid, 
                           scoring = 'r2',
                           cv = 5,
                           n_jobs=-1 # paralellize computation
                          ) 

# Fit data to Grid Search
search1.fit(X_scaled,y_train);

❓ How many models did you trained overall?

<details>
    <summary>Hint</summary>

Much more than 15. Think twice :)
    <details>
    <summary>Answer</summary>

75 models due to CV=5
</details>

In [109]:
75

75

❓ What are the best parameters and the best score?

In [115]:
search1.best_score_ , search1.best_params_

(0.7978142226309175, {'n_neighbors': 10, 'p': 1})

## 6. Random Search

Now let's see if a Random Search can find a better combination with the same number of model fits?
Use `RandomizedSearchCV` to
- Randomly sample `k` from a uniform `randint(1,50)` distribition
- Sample `p` from a list [1,2,3]
- Use the correct number of `n_iter` and `cv` to fit the exact same number of models than in your previous GridSearchCV.

In [118]:
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats

# Instanciate model
model = KNeighborsRegressor()

# Hyperparameter Grid
grid = {'n_neighbors': stats.randint(1, 50), 'p': [1,2,3]}

# Instanciate Grid Search
search = RandomizedSearchCV(model, grid, 
                            scoring='r2',
                            n_iter=75,  # number of draws
                            cv=5, n_jobs=-1)

# Fit data to Grid Search
search.fit(X_scaled, y_train)
search.best_score_

0.8015909185013198

In [119]:
search.best_params_ , search.best_score_

({'n_neighbors': 9, 'p': 1}, 0.8015909185013198)

In [121]:
search.best_estimator_

KNeighborsRegressor(n_neighbors=9, p=1)

## 7. Generalization

👇 This is your final chance to fine-tune your model
- Refine your RandomsearchCV if you wish
- Choose your best model hyper-params and instantiate it
- Re-fit it on the __entire__ train set

In [122]:
model = search.best_estimator_

👇 Time has come to discover our model's performance on the **unseen** test set `X_test`. Compute the r2 score for the test set and save it as `r2_test`.

In [127]:
X_test_scale = X_scaler.transform(X_test)
y_pred = model.predict(X_test_scale)
from sklearn.metrics import r2_score
r2_test = r2_score(y_test, y_pred)
#on veut ramener nos X test vers nos X train qu'on avait scaller au début de l'exercice 

In [107]:
search.best_score_

0.7849247938438436

❓ Would you consider the optimized model to generalize well?

<details><summary>Answer</summary>

Test score may decrease a bit with train set. Probably not more than 5%. This can be due to
- An non-representative train/test split
- A cross-val number too small leading to overfitting the model-tuning phase. The more you cross-validated, the more robust your findings will generalize - but you can't increase cv too much if your dataset is too small as you won't keep enough observations in each fold to be representative.
- Our dataset is very small and our hyperparameter optimization is thus extremely dependent (and overfitting) on our train/test split. Always make sure your dataset is much bigger than the total number of hyperparameter combinations you are trying out!
    
</details>

#### 🧪 Test your code 

In [128]:
from nbresult import ChallengeResult
result = ChallengeResult('r2', 
                         r2_test=r2_test)
result.write()
print(result.check())

============================= test session starts ==============================
platform darwin -- Python 3.8.12, pytest-6.2.5, py-1.10.0, pluggy-1.0.0 -- /Users/selmalopez/.pyenv/versions/lewagon_current/bin/python3
cachedir: .pytest_cache
rootdir: /Users/selmalopez/code/selmalopez/data-challenges/05-ML/05-Model-Tuning/01-Workflow
plugins: dash-2.0.0, anyio-3.3.2
collecting ... collected 1 item

tests/test_r2.py::TestR2::test_r2 PASSED                                 [100%]

============================== 1 passed in 0.24s ===============================


💯 You can commit your code:

git add tests/r2.pickle

git commit -m 'Completed r2 step'

git push origin master


## 🏁 Congratulation. Please push the exercise once completed